In [1]:
!pip install findspark

In [2]:
import findspark
from pyspark.sql import SparkSession
from threading import Thread

In [3]:
findspark.init()

# Create Spark session with Kafka support
spark = SparkSession.builder \
    .appName("KafkaSparkIntegration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

spark

In [5]:
# Function to start the stream
def start_stream():
    global query  # Declare query as a global variable if you want to access it outside this function
    kafka_server = "kafka_00:9094"

    # Read data from Kafka topic
    df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_server) \
        .option("subscribe", "stocks_topic") \
        .load()

    # Assuming the value column in the Kafka topic is a simple string
    df_string = df.selectExpr("CAST(value AS STRING)")

    # Define output path
    output_path = "./test"

    # Write the stream to a file in append mode
    query = df_string.writeStream \
        .outputMode("append") \
        .format("json") \
        .option("path", output_path) \
        .option("checkpointLocation", "./checkpoint") \
        .start()

    # This will run until the stream is stopped or an error occurs
    query.awaitTermination()

# Start the stream in a separate thread
stream_thread = Thread(target=start_stream)
stream_thread.start()

In [6]:
if query.isActive:
    query.stop()